# Tensorflow lite model

Converting the Keras model to lite model for egde devices.

In [ ]:
# importing the libraries
import os
from sys import getsizeof

import h5py
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras


In [2]:
tf.__version__


'2.5.0'

In [3]:
# to get model size
def get_folder_size(folder_path: str):
    size = 0
    for path, dirs, files in os.walk(folder_path):
        for file in files:
            fp = os.path.join(path, file)
            size += os.path.getsize(fp)
    return size


In [ ]:
KERAS_MODEL_NAME = "../models/3_sl_aug_model"


In [5]:
model = keras.models.load_model(KERAS_MODEL_NAME)


In [6]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 16)        64        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0

In [ ]:
print(f"Keras Model Size: {get_folder_size(KERAS_MODEL_NAME)} Bytes")

Keras Model Size: 4877596 Bytes


In [ ]:
# creating test set to calculate model accuracy
import pandas as pd

valid_df = pd.read_csv("../data/asl_data/sign_mnist_valid.csv")

# Separate out our target values
y_test = valid_df['label']
del valid_df['label']

# Separate out our image vectors
x_test = valid_df.values

y_test_numpy = y_test.values
y_test = keras.utils.to_categorical(y_test, 24)

# Normalize our image data
x_test = x_test / 255

x_test = x_test.reshape(-1, 28, 28, 1)


In [9]:
loss, acc = model.evaluate(x_test, y_test)
print(f"loss: {loss}, Accuracy: {acc}")


225/225 [==============================] - 1s 5ms/step - loss: 0.0025 - accuracy: 0.9989
loss: 0.002462963340803981, Accuracy: 0.9988845586776733


## TF Lite Model

In [ ]:
TF_LITE_MODEL_NAME = "../models/5_sl_lite.tflite"


In [11]:
# Model convertor
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_model = tf_lite_converter.convert()


INFO:tensorflow:Assets written to: C:\Users\KUMAR~1.SHA\AppData\Local\Temp\tmp5ciphr5s\assets


In [12]:
# Saving the model
with open(TF_LITE_MODEL_NAME, "wb") as f:
    f.write(tf_lite_model)


In [13]:
# size of the saved model
print(f"Model size: {os.path.getsize(TF_LITE_MODEL_NAME)} Bytes")


Model size: 2248212 Bytes


In [14]:
# Size comparison with keras model
os.path.getsize(TF_LITE_MODEL_NAME) / get_folder_size(KERAS_MODEL_NAME)


0.4609262431738914

## Loading the tflite model

In [15]:
# interpreter for tflite model
interpreter = tf.lite.Interpreter(model_path=TF_LITE_MODEL_NAME)

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


In [16]:
input_details


[{'name': 'conv2d_input',
  'index': 0,
  'shape': array([ 1, 28, 28,  1]),
  'shape_signature': array([-1, 28, 28,  1]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [17]:
output_details


[{'name': 'Identity',
  'index': 33,
  'shape': array([ 1, 24]),
  'shape_signature': array([-1, 24]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [18]:
print(f"Input Shape: {input_details[0]['shape']}")
print(f"Input type: {input_details[0]['dtype']}")
print(f"output Shape: {output_details[0]['shape']}")
print(f"output type: {output_details[0]['dtype']}")


Input Shape: [ 1 28 28  1]
Input type: <class 'numpy.float32'>
output Shape: [ 1 24]
output type: <class 'numpy.float32'>


In [19]:
x_test.dtype


dtype('float64')

In [20]:
x_test_numpy = np.array(x_test, dtype=np.float32)
x_test_numpy.dtype


dtype('float32')

In [21]:
x_test_numpy.shape


(7172, 28, 28, 1)

In [22]:
y_test.shape


(7172, 24)

In [23]:
# resize tensor shape
interpreter.resize_tensor_input(input_details[0]['index'], x_test_numpy.shape)
interpreter.resize_tensor_input(output_details[0]['index'], y_test.shape)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


In [24]:
print(f"Input Shape: {input_details[0]['shape']}")
print(f"Input type: {input_details[0]['dtype']}")
print(f"output Shape: {output_details[0]['shape']}")
print(f"output type: {output_details[0]['dtype']}")


Input Shape: [7172   28   28    1]
Input type: <class 'numpy.float32'>
output Shape: [7172   24]
output type: <class 'numpy.float32'>


In [25]:
interpreter.set_tensor(input_details[0]['index'], x_test_numpy)
interpreter.invoke()
tflite_prediction = interpreter.get_tensor(output_details[0]['index'])


In [26]:
print(f"Prediction results shape: {tflite_prediction.shape}")
prediction_classes = np.argmax(tflite_prediction, axis=1)


Prediction results shape: (7172, 24)


In [27]:
prediction_classes


array([6, 5, 9, ..., 2, 4, 2], dtype=int64)

In [28]:
y_test_numpy


array([6, 5, 9, ..., 2, 4, 2], dtype=int64)

In [ ]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(prediction_classes, y_test_numpy)
print(f"Accuracy of TFLite Model: {acc}")


Accuracy of TFLite Model: 0.9988845510317903


## TFLite Model Float16

converting to more reduced size model by using float 16 instead of float32 

In [30]:
TF_LITE_FLOAT16_MODEL_NAME = "../models/tf_lite_float16.tflite"


In [31]:
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_converter.optimizations = [tf.lite.Optimize.DEFAULT]
tf_lite_converter.target_spec.supported_types = [tf.float16]
tf_lite_model = tf_lite_converter.convert()


INFO:tensorflow:Assets written to: C:\Users\KUMAR~1.SHA\AppData\Local\Temp\tmpx7yvglsw\assets


INFO:tensorflow:Assets written to: C:\Users\KUMAR~1.SHA\AppData\Local\Temp\tmpx7yvglsw\assets


In [32]:
with open(TF_LITE_FLOAT16_MODEL_NAME, "wb") as f:
    f.write(tf_lite_model)


In [33]:
print(f"Model size: {os.path.getsize(TF_LITE_FLOAT16_MODEL_NAME)} Bytes")


Model size: 1129472 Bytes


In [34]:
# size comparison
os.path.getsize(TF_LITE_FLOAT16_MODEL_NAME) / get_folder_size(KERAS_MODEL_NAME)


0.23156325370120853

In [35]:
os.path.getsize(TF_LITE_FLOAT16_MODEL_NAME) / \
    os.path.getsize(TF_LITE_MODEL_NAME)


0.502386785587836

In [36]:
interpreter = tf.lite.Interpreter(model_path=TF_LITE_MODEL_NAME)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# resize tensor shape
interpreter.resize_tensor_input(input_details[0]['index'], x_test_numpy.shape)
interpreter.resize_tensor_input(output_details[0]['index'], y_test.shape)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.set_tensor(input_details[0]['index'], x_test_numpy)
interpreter.invoke()
tflite_prediction = interpreter.get_tensor(output_details[0]['index'])
prediction_classes = np.argmax(tflite_prediction, axis=1)

acc = accuracy_score(prediction_classes, y_test_numpy)
print(f"Accuracy of TFLite Model: {acc}")


Accuracy of TFLite Model: 0.9988845510317903
